In [9]:
import ee
import geemap
import os
import numpy as np
import geopandas as gpd
from shapely.geometry import box
import zipfile
import contextlib

In [10]:
ee.Initialize()

In [19]:
date_beg = '2015-12-01'
date_end = '2017-12-31'

In [20]:
Pl = (ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa')
      .filterDate(date_beg, date_end)
      .select(['B','G','R','N'])
      )

img = Pl.median()
# .toList(Pl.size().getInfo()).get(2)

vis_params = {"min": 0, "max": 3000, "bands": ["R", "G", "B"]}

Map = geemap.Map(zoom = 10)

Map.addLayer(ee.Image(img), vis_params, "NICFI")

# Map.addLayer(aoi_T, vis_params = {'color' : 'red'})

# Map.centerObject(aoi_T, zoom = 8)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## For Tanzania

In [5]:
aoi_T = geemap.shp_to_ee('../Data/Vector/Cashew_Tanzania.shp')

data = aoi_T.geometry().bounds().buffer(1000)

scale = data.projection().nominalScale().getInfo()

img_size = 5120*2*4.77

features = geemap.fishnet(data, h_interval = img_size/scale, v_interval=img_size/scale) #100kmx100km grid

Pl_Tanz = Pl.map(lambda im : im.clip(data))

In [6]:
data.area(maxError=1).divide(1000*1000).getInfo()*100

1562166.6384550477

In [7]:
Map = geemap.Map(zoom = 10)

Map.addLayer(ee.Image(img), vis_params, "NICFI")

Map.addLayer(aoi_T, vis_params = {'color' : 'red'})

Map.centerObject(aoi_T, zoom = 8)

Map

Map(center=[-10.570131338534335, 39.590897292433276], controls=(WidgetControl(options=['position', 'transparen…

In [8]:
geemap.download_ee_image(ee.Image(Pl_Tanz.toList(Pl_Tanz.size().getInfo()).get(2)), '../Data/Planet/TanzaniaStudyArea.tif', crs = 'EPSG:32736', scale = 4.77)

Consider adjusting `region`, `scale` and/or `dtype` to reduce the TanzaniaStudyArea.tif download size (raw: 5.78 GB).


TanzaniaStudyArea.tif: |                                              | 0.00/5.78G (raw) [  0.0%] in 00:00 (et…

## For Ivory Coast

In [5]:
gdf = gpd.read_file('../Data/Vector/Cashew_Ivorycoast.shp').explode()
BBox = gpd.GeoDataFrame(index = [0], crs = 'epsg:4326',geometry = [box(*gdf.dissolve().total_bounds)])

with contextlib.suppress(PermissionError):
    BB = geemap.geopandas_to_ee(BBox)
    data = BB.geometry().bounds().buffer(1000)

Pl_Iv = Pl.map(lambda im : im.clip(data))

scale = data.projection().nominalScale().getInfo()
features = geemap.fishnet(data, h_interval=(51200*5.45)/scale, v_interval = (51200*4.4)/scale) #100kmx100km grid

C:\Users\mdomi\AppData\Local\Temp\ipykernel_22572\2444591401.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  gdf = gpd.read_file('../Data/Vector/Cashew_Ivorycoast.shp').explode()


In [6]:
data.area(maxError=1).divide(1000*1000).getInfo()*100

24118031.786508404

In [7]:
Map = geemap.Map()

Map.addLayer(BB)

Map.addLayer(ee.Image(Pl_Iv.toList(Pl_Iv.size().getInfo()).get(2)), vis_params)

Map.addLayer(features, vis_params = {'color' : 'blue'})

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
with contextlib.suppress(PermissionError):
    geemap.download_ee_image_tiles(
        ee.Image(Pl_Iv.toList(Pl_Iv.size().getInfo()).get(2)), features, '../Data/Planet', prefix="PlanetTilesIvory_", crs="EPSG:2041", scale=4.77
  )

Consider adjusting `region`, `scale` and/or `dtype` to reduce the PlanetTilesIvory_1.tif download size (raw: 22.25 GB).


PlanetTilesIvory_1.tif: |                                             | 0.00/22.2G (raw) [  0.0%] in 00:00 (et…

Consider adjusting `region`, `scale` and/or `dtype` to reduce the PlanetTilesIvory_2.tif download size (raw: 21.91 GB).


PlanetTilesIvory_2.tif: |                                             | 0.00/21.9G (raw) [  0.0%] in 00:00 (et…

Consider adjusting `region`, `scale` and/or `dtype` to reduce the PlanetTilesIvory_3.tif download size (raw: 22.24 GB).


PlanetTilesIvory_3.tif: |                                             | 0.00/22.2G (raw) [  0.0%] in 00:00 (et…

Consider adjusting `region`, `scale` and/or `dtype` to reduce the PlanetTilesIvory_4.tif download size (raw: 21.83 GB).


PlanetTilesIvory_4.tif: |                                             | 0.00/21.8G (raw) [  0.0%] in 00:00 (et…